# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    print(files)
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/michaeleby1/Documents/Flatiron/udacity/2 Data Modeling/sparkify-cassandra
['2018-11-15-events.csv', '2018-11-22-events.csv', '2018-11-09-events.csv', '2018-11-18-events.csv', '2018-11-04-events.csv', '2018-11-01-events.csv', '2018-11-27-events.csv', '2018-11-10-events.csv', '2018-11-20-events.csv', '2018-11-17-events.csv', '2018-11-06-events.csv', '2018-11-03-events.csv', '2018-11-28-events.csv', '2018-11-12-events.csv', '2018-11-25-events.csv', '2018-11-26-events.csv', '2018-11-11-events.csv', '2018-11-14-events.csv', '2018-11-23-events.csv', '2018-11-08-events.csv', '2018-11-19-events.csv', '2018-11-05-events.csv', '2018-11-02-events.csv', '2018-11-29-events.csv', '2018-11-13-events.csv', '2018-11-24-events.csv', '2018-11-21-events.csv', '2018-11-16-events.csv', '2018-11-07-events.csv', '2018-11-30-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

8056


In [5]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
! pip install cassandra-driver

In [8]:
print(cassandra.__version__)

3.23.0


In [1]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace("sparkify")
    
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [37]:
## CQL Queries

create_song_library = """
    CREATE TABLE IF NOT EXISTS 
        song_library (
            session_id int,
            item_in_session int,
            artist text,
            song_title text,
            song_length float,
            PRIMARY KEY (session_id, item_in_session)
        )
"""

insert_song_library = """
    INSERT INTO 
        song_library (
            session_id,
            item_in_session,
            artist,
            song_title,
            song_length
        )
        VALUES
        (%s, %s, %s, %s, %s)
"""

select_song_library = """
    SELECT
        artist,
        song_title,
        song_length
    FROM
        song_library
    WHERE
        session_id = 338
    AND
        item_in_session = 4
"""

In [16]:
## Create table
try:
    session.execute(create_song_library)
except Exception as e:
    print(e)

In [46]:
## Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#         print(line)

        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_song_library, 
                        (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [48]:
## SELECT statement to verify the data was entered into the table
  
rows = session.execute(select_song_library)
df = pd.DataFrame(rows)
df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [38]:
## CQL Queries

create_user_library = """
    CREATE TABLE IF NOT EXISTS 
        user_library (
            user_id int,
            session_id int,
            item_in_session int,
            artist text,
            song_title text,
            first_name text,
            last_name text,
            PRIMARY KEY (user_id, session_id, item_in_session)
        )
"""

insert_user_library = """
    INSERT INTO
        user_library (
            user_id,
            session_id,
            item_in_session,
            artist,
            song_title,
            first_name,
            last_name
        )
        VALUES
        (%s, %s, %s, %s, %s, %s, %s)
"""

select_user_library = """
    SELECT
        artist,
        song_title,
        first_name,
        last_name
    FROM
        user_library
    WHERE
        user_id = 10
    AND
        session_id = 182
"""
                    

In [39]:
## Create table
try:
    session.execute(create_user_library)
except Exception as e:
    print(e)

In [41]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [ ]:
user_id,
            session_id,
            item_in_session,
            artist,
            song_title,
            first_name,
            last_name

In [43]:
## Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#         print(line)
        
        session.execute(insert_user_library, 
                        (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],
                         line[1], line[4]))

In [49]:
## SELECT statement to verify the data was entered into the table
rows = session.execute(select_user_library)
df = pd.DataFrame(rows)
df

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
## CQL Queries

create_music_app_history = """
    CREATE 
"""

insert_music_app_history

select_music_app_history
                   

In [ ]:
## Create table

In [ ]:
## Insert data


In [ ]:
## SELECT statement to verify the data was entered into the table


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()